In [1]:
import pandas as pd
df=pd.read_csv('CosmoCloudHackathonDataset.csv')  
df.head()

,PMID,Title,Abstract,Link-pubmed
0,32361569,"Breast cancer: Biology, biomarkers, and treatm...","During the past recent years, various therapie...",https://pubmed.ncbi.nlm.nih.gov/32361569/
1,27865536,Breast cancer,Breast cancer is one of the three most common ...,https://pubmed.ncbi.nlm.nih.gov/27865536/
2,35243878,"Breast cancer: presentation, investigation and...",Breast cancer is the most common global malign...,https://pubmed.ncbi.nlm.nih.gov/35243878/
3,29471575,[Breast Cancer: Diagnostics and Therapy - the ...,Learning objectives: After reviewing this arti...,https://pubmed.ncbi.nlm.nih.gov/29471575/
4,27009935,Benign and Malignant Breast Disease at Rwanda'...,Background: Breast cancer incidence is rising ...,https://pubmed.ncbi.nlm.nih.gov/27009935/


In [2]:
from document_loading import load_documents_from_df
from document_processing_and_embedding import split_documents, make_document_embeddings

documents = load_documents_from_df(df)
documents = split_documents(documents)

100%|██████████| 30/30 [00:00<00:00, 9027.13it/s]


Documents split into 30 chunks.


In [ ]:
!pip install chromadb 

creating chromadb as vector DB

In [13]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI 
embedding_model = OpenAIEmbeddings()
    
vectorstore = Chroma.from_documents(
    documents=documents,
    collection_name="rag-oncological",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [10]:
retrieved_docs = retriever.invoke("Mutation")
print(len(retrieved_docs))
print(retrieved_docs[0].page_content)

4
Inherited mutations in BRCA1 and BRCA2 lead to significantly increased risks of breast and ovarian cancer. We used epidemiologic methods to evaluate the relative risks of breast cancer vs. ovarian cancer among women of Ashkenazi Jewish ancestry with inherited mutations in BRCA1 or BRCA2. The cancer of a family's index case (i.e., breast cancer vs. ovarian cancer) was significantly associated with site-specific risks of cancer in relatives known to carry mutations in BRCA1 or BRCA2. Specifically, breast cancer risks were higher among relatives of breast cancer index cases compared with relatives of ovarian cancer index cases [hazard ratio (HR) = 3.0, P < 0.001 for BRCA1 carriers and HR = 4.8, P = 0.017 for BRCA2 carriers], and ovarian cancer risks were higher among relatives of ovarian cancer index cases compared with relatives of breast cancer index cases (HR = 7.2, P = 0.001 for BRCA1 carriers and HR = 15.8, P = 0.018 for BRCA2 carriers). Breast and ovarian cancer risks also increas

### For Now I pulled a default rag prompt. We have to refine it for our use case.

In [7]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\utfu\.virtualenvs\OncoSearch-yB5ZPERY\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Creating LLM Chain

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser



llm = ChatOpenAI(model="gpt-4o-mini")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What are some ways to treat breast Cancer")

'Breast cancer treatment options include surgery, radiation therapy, chemotherapy, endocrine therapy, and targeted therapy, depending on the cancer stage and subtype. Early-stage cancers often receive breast-conserving surgery followed by radiation, while more advanced stages may require neoadjuvant chemotherapy to shrink tumors before surgery. For metastatic breast cancer, treatments focus on prolonging survival and maintaining quality of life through systemic therapies.'